# 1. Round Table Problem

Exactly six trade representatives negotiate a treaty: Klosnik, Londi, Manley, Neri, Osata, Poirier. There are exactly six chairs evenly spaced around a circular table. The chairs are numbered 1 through 6, with successively numbered chairs next to each other and chair number 1 next to chair number 6. Each chair is occupied by exactly one of the representatives. The following conditions apply: Poirier sits immediately next to Neri. Londi sits immediately next to Manley, Neri, or both. Klosnik does not sit immediately next to Manley. If Osata sits immediately next to Poirier, Osata does not sit immediately next to Manley.	

Which one of the following seating arrangements of the six representatives in chairs 1 through 6 would NOT violate the stated conditions?	

- "Klosnik, Poirier, Neri, Manley, Osata, Londi", 

- "Klosnik, Londi, Manley, Poirier, Neri, Osata", 

- "Klosnik, Londi, Manley, Osata, Poirier, Neri", 

- "Klosnik, Osata, Poirier, Neri, Londi, Manley", 

- "Klosnik, Neri, Londi, Osata, Manley, Poirier"

### RDFLib implementation

In [21]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD

# Create a new RDF graph
g = Graph()

# Define namespace
NEGO = Namespace("http://example.org/negotiation#")
g.bind("nego", NEGO)

def create_seating_arrangement(arrangement):
    g.remove((None, None, None))  # Clear previous arrangement
    
    # Create chairs and representatives
    for i in range(1, 7):
        chair = URIRef(NEGO[f"Chair{i}"])
        g.add((chair, RDF.type, NEGO.Chair))
        g.add((chair, NEGO.seatNumber, Literal(i, datatype=XSD.integer)))
        
        rep = URIRef(NEGO[arrangement[i-1]])
        g.add((rep, RDF.type, NEGO.Representative))
        g.add((chair, NEGO.occupiedBy, rep))
    
    # Add circular adjacency
    for i in range(1, 7):
        chair = URIRef(NEGO[f"Chair{i}"])
        next_chair = URIRef(NEGO[f"Chair{1 if i == 6 else i+1}"])
        g.add((chair, NEGO.adjacentTo, next_chair))
        g.add((next_chair, NEGO.adjacentTo, chair))

def check_conditions():
    # Condition 1: Poirier sits next to Neri
    q1 = """ASK {
        ?pChair nego:occupiedBy nego:Poirier ;
                nego:adjacentTo/nego:occupiedBy nego:Neri .
    }"""
    
    # Condition 2: Londi sits next to Manley or Neri
    q2 = """ASK {
        ?lChair nego:occupiedBy nego:Londi ;
                nego:adjacentTo/nego:occupiedBy ?neighbor .
        FILTER(?neighbor IN (nego:Manley, nego:Neri))
    }"""
    
    # Condition 3: Klosnik not next to Manley
    q3 = """ASK {
        FILTER NOT EXISTS {
            ?kChair nego:occupiedBy nego:Klosnik ;
                    nego:adjacentTo/nego:occupiedBy nego:Manley .
        }
    }"""
    
    # Condition 4: If Osata-Poirier adjacent, not Osata-Manley
    q4 = """ASK {
        FILTER NOT EXISTS {
            ?osataChair nego:occupiedBy nego:Osata ;
                        nego:adjacentTo/nego:occupiedBy nego:Poirier ;
                        nego:adjacentTo/nego:occupiedBy nego:Manley .
        }
    }"""
    
    return all(g.query(q).askAnswer for q in [q1, q2, q3, q4])

In [22]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD

# Create a new RDF graph
g = Graph()

# Define namespace
NEGO = Namespace("http://example.org/negotiation#")
g.bind("nego", NEGO)

# Example seating arrangement
arrangement = ["Poirier", "Manley", "Londi", "Neri", "Klosnik", "Osata"]

# Create seating arrangement and check conditions
create_seating_arrangement(arrangement)

# Display the graph in Turtle format
print(g.serialize(format='turtle'))

@prefix nego: <http://example.org/negotiation#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

nego:Klosnik a nego:Representative .

nego:Londi a nego:Representative .

nego:Manley a nego:Representative .

nego:Neri a nego:Representative .

nego:Osata a nego:Representative .

nego:Poirier a nego:Representative .

nego:Chair1 a nego:Chair ;
    nego:adjacentTo nego:Chair2,
        nego:Chair6 ;
    nego:occupiedBy nego:Poirier ;
    nego:seatNumber 1 .

nego:Chair2 a nego:Chair ;
    nego:adjacentTo nego:Chair1,
        nego:Chair3 ;
    nego:occupiedBy nego:Manley ;
    nego:seatNumber 2 .

nego:Chair3 a nego:Chair ;
    nego:adjacentTo nego:Chair2,
        nego:Chair4 ;
    nego:occupiedBy nego:Londi ;
    nego:seatNumber 3 .

nego:Chair4 a nego:Chair ;
    nego:adjacentTo nego:Chair3,
        nego:Chair5 ;
    nego:occupiedBy nego:Neri ;
    nego:seatNumber 4 .

nego:Chair5 a nego:Chair ;
    nego:adjacentTo nego:Chair4,
        nego:Chair6 ;
    nego:occupiedBy nego:Klosnik ;

In [23]:
# Test all arrangements
arrangements = [
    ["Klosnik", "Poirier", "Neri", "Manley", "Osata", "Londi"],
    ["Klosnik", "Londi", "Manley", "Poirier", "Neri", "Osata"],
    ["Klosnik", "Londi", "Manley", "Osata", "Poirier", "Neri"],
    ["Klosnik", "Osata", "Poirier", "Neri", "Londi", "Manley"],
    ["Klosnik", "Neri", "Londi", "Osata", "Manley", "Poirier"]
]

for i, arr in enumerate(arrangements, 1):
    create_seating_arrangement(arr)
    valid = check_conditions()
    print(f"Option {i}: {'Valid' if valid else 'Invalid'} - {arr}")

Option 1: Invalid - ['Klosnik', 'Poirier', 'Neri', 'Manley', 'Osata', 'Londi']
Option 2: Valid - ['Klosnik', 'Londi', 'Manley', 'Poirier', 'Neri', 'Osata']
Option 3: Invalid - ['Klosnik', 'Londi', 'Manley', 'Osata', 'Poirier', 'Neri']
Option 4: Invalid - ['Klosnik', 'Osata', 'Poirier', 'Neri', 'Londi', 'Manley']
Option 5: Invalid - ['Klosnik', 'Neri', 'Londi', 'Osata', 'Manley', 'Poirier']


### Python implementation

In [24]:
import itertools

# Define the list of representatives
representatives = ['K', 'L', 'M', 'N', 'O', 'P']

# Define the conditions for validation

# 1. P sits immediately next to N
def condition_1(arrangement):
    return abs(arrangement.index('P') - arrangement.index('N')) == 1 or \
           abs(arrangement.index('P') - arrangement.index('N')) == len(arrangement) - 1

# 2. L sits immediately next to M, N, or both
def condition_2(arrangement):
    return (abs(arrangement.index('L') - arrangement.index('M')) == 1 or
            abs(arrangement.index('L') - arrangement.index('N')) == 1)

# 3. K does not sit immediately next to M
def condition_3(arrangement):
    pos_K = arrangement.index('K')
    pos_M = arrangement.index('M')
    # Check if K and M are next to each other in a circular manner
    return abs(pos_K - pos_M) != 1 and abs(pos_K - pos_M) != len(arrangement) - 1

# 4. If O sits immediately next to P, O does not sit immediately next to M
def condition_4(arrangement):
    pos_O = arrangement.index('O')
    pos_P = arrangement.index('P')
    pos_M = arrangement.index('M')
    # Check if O and P are next to each other
    if abs(pos_O - pos_P) == 1 or abs(pos_O - pos_P) == len(arrangement) - 1:
        # If O and P are next to each other, O should not be next to M
        if abs(pos_O - pos_M) == 1 or abs(pos_O - pos_M) == len(arrangement) - 1:
            return False
    return True

# Helper function to normalize circular permutations
def normalize(arrangement):
    """Returns the lexicographically smallest rotation of the arrangement."""
    rotations = [arrangement[i:] + arrangement[:i] for i in range(len(arrangement))]
    return min(rotations)

# Generate all permutations of the seating arrangement
all_arrangements = itertools.permutations(representatives)

# Set to store unique valid arrangements (ignoring rotations)
unique_valid_arrangements = set()

# Iterate through all possible seating arrangements and check validity
for arrangement in all_arrangements:
    if (condition_1(arrangement) and condition_2(arrangement) and
        condition_3(arrangement) and condition_4(arrangement)):
        # Normalize and add to the set of unique valid arrangements
        normalized_arrangement = normalize(arrangement)
        unique_valid_arrangements.add(normalized_arrangement)

# Display unique valid seating arrangements
if unique_valid_arrangements:
    print("Valid seating arrangements found:", len(unique_valid_arrangements))
    for valid in sorted(unique_valid_arrangements):
        print(valid)
else:
    print("No valid seating arrangements found.")

# Check if a given arrangement is valid based on the unique_valid_arrangements set
def is_valid_arrangement(arrangement, valid_set):
    # Normalize the arrangement to its lexicographically smallest rotation
    normalized_arrangement = normalize(arrangement)
    # Check if the normalized arrangement is in the set of valid arrangements
    return normalized_arrangement in valid_set

Valid seating arrangements found: 16
('K', 'L', 'M', 'N', 'P', 'O')
('K', 'L', 'M', 'O', 'N', 'P')
('K', 'L', 'M', 'P', 'N', 'O')
('K', 'L', 'N', 'P', 'M', 'O')
('K', 'N', 'P', 'L', 'M', 'O')
('K', 'N', 'P', 'M', 'L', 'O')
('K', 'O', 'L', 'M', 'N', 'P')
('K', 'O', 'L', 'M', 'P', 'N')
('K', 'O', 'M', 'L', 'N', 'P')
('K', 'O', 'M', 'L', 'P', 'N')
('K', 'O', 'M', 'P', 'N', 'L')
('K', 'O', 'N', 'P', 'M', 'L')
('K', 'O', 'P', 'N', 'M', 'L')
('K', 'P', 'N', 'L', 'M', 'O')
('K', 'P', 'N', 'M', 'L', 'O')
('K', 'P', 'N', 'O', 'M', 'L')


In [25]:
# Define the list of sample arrangements to check
sample_arrangements = [
    ('K', 'P', 'N', 'M', 'O', 'L'),
    ('K', 'L', 'M', 'P', 'N', 'O'),
    ('K', 'L', 'M', 'O', 'P', 'N'),
    ('K', 'O', 'P', 'N', 'L', 'M'),
    ('K', 'N', 'L', 'O', 'M', 'P')
]

# Check each arrangement and print the result
for arrangement in sample_arrangements:
    if is_valid_arrangement(arrangement, unique_valid_arrangements):
        print(f"The arrangement {arrangement} is valid.")
    else:
        print(f"The arrangement {arrangement} is NOT valid.")

The arrangement ('K', 'P', 'N', 'M', 'O', 'L') is NOT valid.
The arrangement ('K', 'L', 'M', 'P', 'N', 'O') is valid.
The arrangement ('K', 'L', 'M', 'O', 'P', 'N') is NOT valid.
The arrangement ('K', 'O', 'P', 'N', 'L', 'M') is NOT valid.
The arrangement ('K', 'N', 'L', 'O', 'M', 'P') is NOT valid.


# 2. Probability of an Even Sum in Four Dice Rolls 

You roll exactly four fair six-sided dice. Each die can land on one of six faces, numbered 1 to 6. Instead of considering the exact values, you classify each roll as follows:  

- A roll is Even if the die lands on 2, 4, or 6.  
- A roll is Odd if the die lands on 1, 3, or 5.  

What is the probability that the sum of the four dice is Even?


### RDFLib implementation

In [26]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD

# Create an RDF graph
g = Graph()

# Define namespace
DICE = Namespace("http://example.org/dice#")
g.bind("dice", DICE)

def create_dice_graph():
    """Creates all possible dice roll outcomes using binary even/odd classification."""
    g.remove((None, None, None))  # Clear previous data
    
    total_rolls = 0
    even_sum_count = 0
    
    # Iterate through all 4-dice binary roll combinations (Even/Odd)
    for d1 in ["Even", "Odd"]:
        for d2 in ["Even", "Odd"]:
            for d3 in ["Even", "Odd"]:
                for d4 in ["Even", "Odd"]:
                    total_rolls += 1
                    roll_id = f"Roll{total_rolls}"
                    roll_uri = URIRef(DICE[roll_id])
                    
                    # Add roll instance
                    g.add((roll_uri, RDF.type, DICE.DiceRoll))
                    
                    # Store dice values as binary logic
                    g.add((roll_uri, DICE.die1, Literal(d1)))
                    g.add((roll_uri, DICE.die2, Literal(d2)))
                    g.add((roll_uri, DICE.die3, Literal(d3)))
                    g.add((roll_uri, DICE.die4, Literal(d4)))
                    
                    # Count how many dice are odd
                    odd_count = sum(1 for die in [d1, d2, d3, d4] if die == "Odd")
                    
                    # Sum is even if odd_count is 0, 2, or 4
                    is_even = odd_count % 2 == 0
                    g.add((roll_uri, DICE.hasEvenSum, Literal(is_even, datatype=XSD.boolean)))
                    
                    if is_even:
                        even_sum_count += 1

    return total_rolls, even_sum_count

def compute_probability():
    """Runs an RDF query to count cases where the sum is even and computes probability."""
    
    query = """
    SELECT (COUNT(?roll) AS ?evenRolls)
    WHERE {
        ?roll a dice:DiceRoll ;
              dice:hasEvenSum true .
    }
    """
    
    result = g.query(query)
    
    # Convert SPARQLResult to list and extract the first value
    even_rolls = int(list(result)[0][0])

    total_rolls, _ = create_dice_graph()  # Ensure graph is populated
    probability = even_rolls / total_rolls
    
    return probability

# Populate RDF Graph
total_rolls, even_sum_count = create_dice_graph()
probability = compute_probability()

print(f"Probability of Sum Being Even: {probability:.4f}")

Probability of Sum Being Even: 0.5000


In [11]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, OWL

# Create namespace
DICE = Namespace("http://example.org/dice#")
g = Graph()

def generate_dice_graph():
    # Define individuals and their properties
    p = URIRef(DICE['p'])
    q = URIRef(DICE['q'])
    r = URIRef(DICE['r'])
    s = URIRef(DICE['s'])
    t = URIRef(DICE['t'])
    u = URIRef(DICE['u'])
    v = URIRef(DICE['v'])
    w = URIRef(DICE['w'])

    g.add((p, RDF.type, DICE.Person))
    g.add((q, RDF.type, DICE.Person))
    g.add((r, RDF.type, DICE.Person))
    g.add((s, RDF.type, DICE.Person))
    g.add((t, RDF.type, DICE.Person))
    g.add((u, RDF.type, DICE.Person))
    g.add((v, RDF.type, DICE.Person))
    g.add((w, RDF.type, DICE.Person))

    # Add properties for circular arrangement
    g.add((p, OWL.sameAs, u))  # p sits second to the right of U
    g.add((u, OWL.inverseOf, p))  # U sits opposite to T
    g.add((t, OWL.inverseOf, u))  # T is opposite to U
    
    # Q and T have two persons between them
    g.add((q, OWL.sameAs, t))  # Two persons sit between Q and T

    # S sits third to the right of P
    g.add((p, OWL.nextInList, s))
    g.add((s, OWL.nextInList, r))

    # V sits second to the left of W
    g.add((v, OWL.prevInList, w))

def count_persons_between_w_and_r():
    generate_dice_graph()

    # Use reasoning engine to infer persons between W and R
    query = """
    SELECT (COUNT(?x) AS ?count)
    WHERE {
        { :w owl:nextInList* ?x . }
        { ?x owl:prevInList* :r .
            FILTER NOT EXISTS {
                ?y owl:sameAs ?x .
                ?y owl:sameAs :p .
                ?z owl:sameAs :s
            }
        }
    }
    """

    result = g.query(query)

    # Extract and return count of persons between W and R
    count = int(list(result)[0][0])

    return "Two" if count == 2 else None

print(count_persons_between_w_and_r())

AttributeError: term 'nextInList' not in namespace 'http://www.w3.org/2002/07/owl#'

### Python implementation

In [27]:
from itertools import product

def probability_even_sum():
    """Computes the probability that the sum of four dice rolls is even using binary logic (Even/Odd)."""
    
    total_rolls = 0
    even_sum_count = 0
    
    # Generate all possible sequences of 4 dice rolls (Even/Odd)
    for roll in product(["Even", "Odd"], repeat=4):
        total_rolls += 1
        odd_count = sum(1 for die in roll if die == "Odd")
        
        # Sum is even if odd_count is 0, 2, or 4
        if odd_count % 2 == 0:
            even_sum_count += 1

    return even_sum_count / total_rolls

# Compute and print probability
prob = probability_even_sum()
print(f"Probability of Sum Being Even: {prob:.4f}")

Probability of Sum Being Even: 0.5000


# 3. Probability of Drawing Three Cards in Ascending Order

You randomly draw exactly three cards from a standard 52-card deck. For simplicity, we will only consider the ranks of the cards, not the suits. The ranks are numbered from 2 to 10, with face cards J (Jack), Q (Queen), K (King), and A (Ace).

What is the probability that the three cards drawn are in ascending order based on their ranks?

### RDFLib implementation

In [28]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD
from itertools import permutations

# Create an RDF graph
g = Graph()

# Define namespace
CARD = Namespace("http://example.org/card#")
g.bind("card", CARD)

def create_card_graph():
    """Creates all possible 3-card combinations and checks if they are in ascending order."""
    g.remove((None, None, None))  # Clear previous data
    
    # Define a standard deck of cards (values from 1 to 13, representing Ace to King)
    deck = list(range(1, 14))
    
    total_combinations = 0
    ascending_count = 0
    
    # Generate all possible 3-card combinations (order matters)
    for cards in permutations(deck, 3):
        total_combinations += 1
        combo_id = f"Combo{total_combinations}"
        combo_uri = URIRef(CARD[combo_id])
        
        # Add combo instance
        g.add((combo_uri, RDF.type, CARD.CardCombo))
        
        # Store card values
        g.add((combo_uri, CARD.card1, Literal(cards[0], datatype=XSD.integer)))
        g.add((combo_uri, CARD.card2, Literal(cards[1], datatype=XSD.integer)))
        g.add((combo_uri, CARD.card3, Literal(cards[2], datatype=XSD.integer)))
        
        # Check if the cards are in ascending order
        is_ascending = cards[0] < cards[1] < cards[2]
        g.add((combo_uri, CARD.isAscending, Literal(is_ascending, datatype=XSD.boolean)))
        
        if is_ascending:
            ascending_count += 1

    return total_combinations, ascending_count

def compute_probability():
    """Runs an RDF query to count ascending combinations and computes probability."""
    
    query = """
    SELECT (COUNT(?combo) AS ?ascendingCombos)
    WHERE {
        ?combo a card:CardCombo ;
               card:isAscending true .
    }
    """
    
    result = g.query(query)
    
    # Convert SPARQLResult to list and extract the first value
    ascending_combos = int(list(result)[0][0])

    total_combinations, _ = create_card_graph()  # Ensure graph is populated
    probability = ascending_combos / total_combinations
    
    return probability

# Populate RDF Graph
total_combinations, ascending_count = create_card_graph()
probability = compute_probability()

print(f"Probability of Drawing Three Cards in Ascending Order: {probability:.4f}")

Probability of Drawing Three Cards in Ascending Order: 0.1667


### Python implementation

In [29]:
from itertools import permutations

def compute_probability():
    """Computes the probability of drawing three cards in ascending order."""
    
    # Define a standard deck of cards (values from 1 to 13, representing Ace to King)
    deck = list(range(1, 14))
    
    total_combinations = 0
    ascending_count = 0
    
    # Generate all possible 3-card permutations (order matters)
    for cards in permutations(deck, 3):
        total_combinations += 1
        
        # Check if the cards are in ascending order
        is_ascending = cards[0] < cards[1] < cards[2]
        
        if is_ascending:
            ascending_count += 1
    
    # Calculate the probability
    probability = ascending_count / total_combinations
    
    return probability

# Compute the probability
probability = compute_probability()

print(f"Probability of Drawing Three Cards in Ascending Order: {probability:.4f}")

Probability of Drawing Three Cards in Ascending Order: 0.1667


# 4. Round Table Problem (Colored Chairs)

Six chairs are arranged around a circular table. Each chair is painted one of three colors: Red, Green, or Blue. The following conditions apply:

No two adjacent chairs can have the same color.

Exactly two chairs are Red.

Exactly two chairs are Green.

Exactly two chairs are Blue.

How many distinct ways can the chairs be painted without violating the conditions?

### RDFLib implementation

In [3]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD
from itertools import permutations

# Create an RDF graph
g = Graph()

# Define namespace
CHAIR = Namespace("http://example.org/chair#")
g.bind("chair", CHAIR)

# Define colors
colors = ["Red", "Green", "Blue"]

# Constraint: Exactly two of each color
valid_combinations = [
    p for p in set(permutations(colors * 2, 6))  # Generate all permutations
    if p.count("Red") == 2 and p.count("Green") == 2 and p.count("Blue") == 2
]

def create_chair_graph():
    """Creates all valid chair arrangements that meet the constraints."""
    g.remove((None, None, None))  # Clear previous data
    
    valid_count = 0
    for idx, arrangement in enumerate(valid_combinations, 1):
        # Ensure no adjacent chairs have the same color
        if any(arrangement[i] == arrangement[(i+1) % 6] for i in range(6)):
            continue  # Skip invalid cases
        
        valid_count += 1
        arrangement_id = f"Arrangement{valid_count}"
        arrangement_uri = URIRef(CHAIR[arrangement_id])
        
        # Add arrangement instance
        g.add((arrangement_uri, RDF.type, CHAIR.ChairArrangement))
        
        # Assign chair colors
        for i, color in enumerate(arrangement):
            chair_uri = URIRef(CHAIR[f"chair{i+1}"])
            g.add((arrangement_uri, CHAIR.hasChair, chair_uri))
            g.add((chair_uri, CHAIR.hasColor, Literal(color)))
    
    return valid_count

def compute_valid_arrangements():
    """Runs an RDF query to count valid arrangements."""
    query = """
    SELECT (COUNT(?arrangement) AS ?validArrangements)
    WHERE {
        ?arrangement a chair:ChairArrangement .
    }
    """
    
    result = g.query(query)
    
    # Convert SPARQLResult to list and extract the first value
    valid_arrangements = int(list(result)[0][0])
    
    return valid_arrangements

# Populate RDF Graph
valid_count = create_chair_graph()
valid_arrangements = compute_valid_arrangements()

print(f"Number of Valid Chair Arrangements: {valid_arrangements}")

Number of Valid Chair Arrangements: 24


### Python implementation

In [31]:
from itertools import permutations

# Define colors
colors = ["Red", "Green", "Blue"]

# Constraint: Exactly two of each color
valid_combinations = [
    p for p in set(permutations(colors * 2, 6))  # Generate all unique permutations
    if p.count("Red") == 2 and p.count("Green") == 2 and p.count("Blue") == 2
]

def count_valid_arrangements():
    """Counts valid chair arrangements where no two adjacent chairs have the same color."""
    valid_count = 0

    for arrangement in valid_combinations:
        # Check that no adjacent chairs have the same color
        if any(arrangement[i] == arrangement[(i+1) % 6] for i in range(6)):
            continue  # Skip invalid cases
        
        valid_count += 1  # Valid arrangement found

    return valid_count

# Compute the number of valid arrangements
valid_arrangements = count_valid_arrangements()
print(f"Number of Valid Chair Arrangements: {valid_arrangements}")

Number of Valid Chair Arrangements: 24


# 5. Puzzle: The Bridge Crossing

Four people need to cross a rickety bridge at night. They have one flashlight, and the bridge can only support two people at a time. The four people walk at different speeds: Person A takes 1 minute, Person B takes 2 minutes, Person C takes 5 minutes, and Person D takes 10 minutes. When two people cross together, they must move at the slower person's pace. What is the minimum total time required for all four people to cross the bridge?

### RDFLib implementation

In [32]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD

# Create an RDF graph
g = Graph()

# Define namespace
BRIDGE = Namespace("http://example.org/bridge#")
g.bind("bridge", BRIDGE)

# Define people and their crossing times
people = {"A": 1, "B": 2, "C": 5, "D": 10}


def calculate_min_crossing_time():
    """Computes the minimum crossing time using an optimal strategy dynamically."""
    times = sorted(people.values())  # Sort times in ascending order
    total_time = 0
    
    while len(times) > 3:
        # Two possible strategies:
        # 1. Send the two fastest first, bring one back, send the two slowest, bring one back
        option1 = times[1] + times[0] + times[-1] + times[1]
        # 2. Send the two slowest, bring one back, send the two fastest, bring one back
        option2 = times[-1] + times[0] + times[-2] + times[0]
        
        total_time += min(option1, option2)
        times.pop()  # Remove the slowest person
        times.pop()  # Remove the second slowest person
    
    if len(times) == 3:
        total_time += sum(times)  # Remaining three cross together optimally
    elif len(times) == 2:
        total_time += max(times)  # Only two remain, take the maximum time
    
    return total_time


def create_bridge_graph():
    """Creates the RDF representation of the optimal crossing sequence dynamically."""
    g.remove((None, None, None))  # Clear previous data
    total_time = calculate_min_crossing_time()
    
    # Store in RDF graph
    crossing_uri = URIRef(BRIDGE.Crossing)
    g.add((crossing_uri, RDF.type, BRIDGE.BridgeCrossing))
    g.add((crossing_uri, BRIDGE.totalTime, Literal(total_time, datatype=XSD.integer)))
    
    return total_time


def compute_minimum_time():
    """Runs an RDF query to get the minimum crossing time."""
    query = """
    SELECT ?totalTime WHERE {
        ?crossing a bridge:BridgeCrossing ;
                  bridge:totalTime ?totalTime .
    }
    """
    result = g.query(query)
    
    # Extract the total time from SPARQL result
    min_time = int(list(result)[0][0])
    return min_time


# Populate RDF Graph
minimum_time = create_bridge_graph()
computed_time = compute_minimum_time()

print(f"Minimum Total Time for Crossing: {computed_time} minutes")

Minimum Total Time for Crossing: 17 minutes


### Python implementation

In [33]:
def min_crossing_time(people):
    """Computes the minimum crossing time using an optimal strategy."""
    times = sorted(people.values())  # Sort times in ascending order
    total_time = 0

    while len(times) > 3:
        # Two possible strategies:
        # 1. Send the two fastest first, bring one back, send the two slowest, bring one back
        option1 = times[1] + times[0] + times[-1] + times[1]
        # 2. Send the two slowest, bring one back, send the two fastest, bring one back
        option2 = times[-1] + times[0] + times[-2] + times[0]

        total_time += min(option1, option2)
        times.pop()  # Remove the slowest person
        times.pop()  # Remove the second slowest person

    if len(times) == 3:
        total_time += sum(times)  # Remaining three cross together optimally
    elif len(times) == 2:
        total_time += max(times)  # Only two remain, take the maximum time

    return total_time


# Define people and their crossing times
people = {"A": 1, "B": 2, "C": 5, "D": 10}

# Compute the minimum time for all to cross
minimum_time = min_crossing_time(people)

print(f"Minimum Total Time for Crossing: {minimum_time} minutes")

Minimum Total Time for Crossing: 17 minutes


# 6. Puzzle: The Poisoned Wine

You have 1000 bottles of wine, one of which is poisoned. You also have 10 rats to test the wine. If the 2nd, 5th and 7th rat die, which bottle contains the poison?

### RDFLib implementation

In [34]:
# 6. Puzzle: The Poisoned Wine
# You have 1000 bottles of wine, one of which is poisoned. You also have 10 rats to test the wine. The poison takes exactly 1 hour to kill a rat, and you only have 1 hour to 
# determine which bottle is poisoned. How do you determine the poisoned bottle using the rats?

from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD

# Create an RDF graph
g = Graph()

# Define namespace
WINE = Namespace("http://example.org/wine#")
g.bind("wine", WINE)

# Define bottles and rats
num_bottles = 1000
num_rats = 10  # Each rat represents a bit in a 10-bit number

def create_wine_graph():
    """Creates RDF representation of bottles, rats, and their drinking pattern."""
    g.remove((None, None, None))  # Clear previous data
    
    # Assign each bottle a unique 10-bit binary encoding
    for bottle in range(1, num_bottles + 1):
        bottle_uri = URIRef(WINE[f"Bottle{bottle}"])
        g.add((bottle_uri, RDF.type, WINE.Bottle))
        
        # Convert bottle number to 10-bit binary
        binary_repr = format(bottle, '010b')
        
        for rat in range(num_rats):
            rat_uri = URIRef(WINE[f"Rat{rat+1}"])
            g.add((rat_uri, RDF.type, WINE.Rat))
            
            # If the bit is 1, the rat drinks from this bottle
            if binary_repr[rat] == '1':
                g.add((bottle_uri, WINE.isTestedBy, rat_uri))

    return num_bottles, num_rats

def determine_poisoned_bottle(dead_rats):
    """Determines the poisoned bottle by interpreting dead rats' binary pattern."""
    
    # Convert dead rats list to binary representation (rat 1 -> most significant bit)
    poisoned_binary = ''.join('1' if f"Rat{i+1}" in dead_rats else '0' for i in range(num_rats))
    
    # Convert binary to decimal (bottle number)
    poisoned_bottle = int(poisoned_binary, 2)
    
    return poisoned_bottle


# Populate RDF Graph
create_wine_graph()

# Example: Suppose Rats 2, 5, and 7 die, corresponding to binary '0100100100'
dead_rats = ["Rat2", "Rat5", "Rat7"]
poisoned_bottle = determine_poisoned_bottle(dead_rats)

print(f"Poisoned Bottle Identified: Bottle {poisoned_bottle}")

Poisoned Bottle Identified: Bottle 296


### Python implementation

In [35]:
def assign_bottles_to_rats(num_bottles=1000, num_rats=10):
    """
    Assigns bottles to rats based on binary encoding.
    
    Returns:
    - rat_drinks: A dictionary where each rat (0-9) has a list of bottle numbers it drinks from.
    - bottle_to_binary: A dictionary mapping each bottle to its 10-bit binary representation.
    """
    rat_drinks = {i: [] for i in range(num_rats)}
    bottle_to_binary = {}

    for bottle in range(1, num_bottles + 1):
        binary_repr = format(bottle, '010b')  # Convert bottle number to 10-bit binary
        bottle_to_binary[bottle] = binary_repr
        
        for rat in range(num_rats):
            if binary_repr[rat] == '1':  # If bit is 1, rat drinks from this bottle
                rat_drinks[rat].append(bottle)

    return rat_drinks, bottle_to_binary


def find_poisoned_bottle(dead_rats):
    """
    Determines the poisoned bottle by decoding the dead rats' binary pattern.
    
    Args:
    - dead_rats: A list of rat indices (0-9) that died after 1 hour.

    Returns:
    - The poisoned bottle number.
    """
    poisoned_binary = ['0'] * 10  # Initialize a 10-bit binary list
    
    for rat in dead_rats:
        poisoned_binary[rat - 1] = '1'  # Mark the bit positions where rats died

    poisoned_bottle = int(''.join(poisoned_binary), 2)  # Convert binary to decimal
    return poisoned_bottle


# Assign bottles to rats
rat_drinks, bottle_to_binary = assign_bottles_to_rats()

# Example: Suppose Rats 2, 5, and 7 die, corresponding to binary '0100100100'
dead_rats = [2, 5, 7]
poisoned_bottle = find_poisoned_bottle(dead_rats)

print(f"Poisoned Bottle Identified: Bottle {poisoned_bottle}")

Poisoned Bottle Identified: Bottle 296


# 7. Puzzle: Monty Hall

A game show features three closed doors: Door 1, Door 2, and Door 3. Behind one door is a car (the prize), and behind the other two doors are goats. A contestant randomly selects one of the doors. The host, who knows where the car is, then opens one of the remaining two doors, always revealing a goat. The contestant is then given the choice to either stay with their original door or switch to the remaining unopened door.

- What is the probability that the contestant wins the car if they decide to stay with their original choice?
- What is the probability that the contestant wins the car if they switch to the other door?

### RDFLib implementation

In [36]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD

# Define namespace
MONTY = Namespace("http://example.org/monty#")

# Create RDF Graph
g = Graph()
g.bind("monty", MONTY)

# Define doors
doors = [URIRef(MONTY[f"Door{i}"]) for i in range(1, 4)]
for door in doors:
    g.add((door, RDF.type, MONTY.Door))

# Define all possible cases
cases = [
    {"prize": doors[0], "chosen": doors[0], "revealed": doors[1]},  # Case 1
    {"prize": doors[0], "chosen": doors[1], "revealed": doors[2]},  # Case 2
    {"prize": doors[0], "chosen": doors[2], "revealed": doors[1]},  # Case 3
    {"prize": doors[1], "chosen": doors[0], "revealed": doors[2]},  # Case 4
    {"prize": doors[1], "chosen": doors[1], "revealed": doors[0]},  # Case 5
    {"prize": doors[1], "chosen": doors[2], "revealed": doors[0]},  # Case 6
    {"prize": doors[2], "chosen": doors[0], "revealed": doors[1]},  # Case 7
    {"prize": doors[2], "chosen": doors[1], "revealed": doors[0]},  # Case 8
    {"prize": doors[2], "chosen": doors[2], "revealed": doors[1]},  # Case 9
]

# Add cases to the RDF graph
for idx, case in enumerate(cases):
    scenario = URIRef(MONTY[f"Scenario{idx+1}"])
    g.add((scenario, RDF.type, MONTY.Scenario))
    g.add((scenario, MONTY.hasPrize, case["prize"]))
    g.add((scenario, MONTY.isChosen, case["chosen"]))
    g.add((scenario, MONTY.isRevealed, case["revealed"]))

# SPARQL query to count winning cases for staying
stay_query = """
PREFIX monty: <http://example.org/monty#>
SELECT (COUNT(*) AS ?stayWins) WHERE {
    ?scenario a monty:Scenario ;
              monty:hasPrize ?prize ;
              monty:isChosen ?prize .
}
"""

# SPARQL query to count winning cases for switching
switch_query = """
PREFIX monty: <http://example.org/monty#>
SELECT (COUNT(*) AS ?switchWins) WHERE {
    ?scenario a monty:Scenario ;
              monty:hasPrize ?prize ;
              monty:isChosen ?chosen ;
              monty:isRevealed ?revealed .
    FILTER(?prize != ?chosen && ?prize != ?revealed)
}
"""

# Execute queries
stay_wins = int(list(g.query(stay_query))[0][0])
switch_wins = int(list(g.query(switch_query))[0][0])
total_cases = len(cases)

# Compute probabilities
stay_prob = stay_wins / total_cases
switch_prob = switch_wins / total_cases

# Print results
print(f"Probability of winning if staying: {stay_prob:.4f}")
print(f"Probability of winning if switching: {switch_prob:.4f}")

Probability of winning if staying: 0.3333
Probability of winning if switching: 0.6667


In [37]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD
import random

# Define namespace
MONTY = Namespace("http://example.org/monty#")

def create_monty_hall_scenario():
    """Creates an RDF graph for the Monty Hall problem."""
    g = Graph()
    g.bind("monty", MONTY)

    # Define doors
    doors = [URIRef(MONTY[f"Door{i}"]) for i in range(1, 4)]
    for door in doors:
        g.add((door, RDF.type, MONTY.Door))
    
    # Randomly assign the prize to one door
    prize_door = random.choice(doors)
    g.add((prize_door, MONTY.hasPrize, Literal("Car", datatype=XSD.string)))
    
    # Assign goats to other doors
    for door in doors:
        if door != prize_door:
            g.add((door, MONTY.hasPrize, Literal("Goat", datatype=XSD.string)))
    
    # Contestant's initial choice
    contestant_choice = random.choice(doors)
    g.add((contestant_choice, MONTY.isChosen, Literal(True, datatype=XSD.boolean)))
    
    # Host opens a door with a goat that is not the contestant's choice
    opened_door = None
    for door in doors:
        if door != contestant_choice and door != prize_door:
            g.add((door, MONTY.isOpened, Literal(True, datatype=XSD.boolean)))
            opened_door = door
            break
    
    # Find the remaining unopened door to switch to
    switch_door = next(d for d in doors if d != contestant_choice and d != opened_door)
    
    return g, contestant_choice, prize_door, switch_door

def run_monty_hall_simulation(trials=10000):
    """Simulates the Monty Hall problem and calculates probabilities."""
    win_if_stay = 0
    win_if_switch = 0
    
    for _ in range(trials):
        g, contestant_choice, prize_door, switch_door = create_monty_hall_scenario()
        
        # Staying means keeping the original choice
        if contestant_choice == prize_door:
            win_if_stay += 1
        
        # Switching means choosing the other remaining door
        if switch_door == prize_door:
            win_if_switch += 1
    
    stay_prob = win_if_stay / trials
    switch_prob = win_if_switch / trials
    
    return stay_prob, switch_prob

# Run the simulation
stay_probability, switch_probability = run_monty_hall_simulation()
print(f"Probability of winning if staying: {stay_probability:.4f}")
print(f"Probability of winning if switching: {switch_probability:.4f}")

Probability of winning if staying: 0.3363
Probability of winning if switching: 0.6637


### Python implementation

In [38]:
import random

def monty_hall_simulation(trials=10000):
    win_if_stay = 0
    win_if_switch = 0

    for _ in range(trials):
        doors = [1, 2, 3]  # Three doors
        prize_door = random.choice(doors)  # Place the car behind one door
        contestant_choice = random.choice(doors)  # Contestant picks a door

        # Monty reveals a goat door (not the contestant's choice or prize door)
        remaining_doors = [d for d in doors if d != contestant_choice and d != prize_door]
        monty_opens = random.choice(remaining_doors)

        # The door available for switching
        switch_door = next(d for d in doors if d != contestant_choice and d != monty_opens)

        # Check outcomes
        if contestant_choice == prize_door:
            win_if_stay += 1  # Win if staying
        if switch_door == prize_door:
            win_if_switch += 1  # Win if switching

    # Compute probabilities
    stay_prob = win_if_stay / trials
    switch_prob = win_if_switch / trials

    return stay_prob, switch_prob

# Run the simulation
stay_probability, switch_probability = monty_hall_simulation(10000)
print(f"Winning probability if staying: {stay_probability:.4f}")
print(f"Winning probability if switching: {switch_probability:.4f}")


Winning probability if staying: 0.3330
Winning probability if switching: 0.6670


# 8. Puzzle: Birthday Paradox

At what number of people in a room does the probability of at least two people sharing the same birthday exceed 0.5?

### RDFLib implementation

In [39]:
import random
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF

# Create an RDF graph
g = Graph()

# Define namespaces
BIRTHDAY = Namespace("http://example.org/birthday#")
g.bind("birthday", BIRTHDAY)

def create_people_and_birthdays(num_people):
    """Creates RDF graph with people and their birthdays."""
    g.remove((None, None, None))  # Clear previous data
    birthdays = []
    
    # Generate birthdays for people
    for person_num in range(1, num_people + 1):
        person_uri = URIRef(BIRTHDAY[f"Person{person_num}"])
        birthday = random.randint(1, 365)  # Assign random birthday between 1 and 365
        birthdays.append(birthday)
        
        # Add person and their birthday to the graph
        g.add((person_uri, RDF.type, BIRTHDAY.Person))
        g.add((person_uri, BIRTHDAY.hasBirthday, Literal(birthday)))
    
    return birthdays

def check_for_birthday_collision():
    """Checks for birthday collisions logically."""
    # Retrieve all birthdays
    birthdays = []
    for person_uri in g.subjects(RDF.type, BIRTHDAY.Person):
        birthday = g.value(person_uri, BIRTHDAY.hasBirthday)
        birthdays.append(birthday)
    
    # Check for duplicate birthdays
    seen = set()
    for birthday in birthdays:
        if birthday in seen:
            return True  # Collision found
        seen.add(birthday)
    
    return False  # No collision

def calculate_probability_of_collision(num_people):
    """Calculates the logical probability of a birthday collision."""
    # We can derive the probability of a collision using the complement principle.
    # The formula for the probability of at least two people sharing a birthday is:
    # P(collision) = 1 - P(no collision)
    # P(no collision) = (365/365) * (364/365) * ... * (365 - n + 1) / 365
    
    if num_people > 365:
        return 1  # With more than 365 people, a collision is certain.
    
    probability_no_collision = 1
    for i in range(num_people):
        probability_no_collision *= (365 - i) / 365
    
    probability_collision = 1 - probability_no_collision
    return probability_collision

# Loop to find when probability exceeds 0.5
num_people = 1
while True:
    create_people_and_birthdays(num_people)
    probability = calculate_probability_of_collision(num_people)
    
    if probability > 0.5:
        print(f"With {num_people} people, the probability of a birthday collision exceeds 0.5 ({probability:.4f}).")
        break
    
    num_people += 1

With 23 people, the probability of a birthday collision exceeds 0.5 (0.5073).


### Python implementation

In [40]:
import random

def generate_birthdays(num_people):
    """Generates random birthdays for a group of people."""
    birthdays = []
    for _ in range(num_people):
        birthday = random.randint(1, 365)  # Assign random birthday between 1 and 365
        birthdays.append(birthday)
    return birthdays

def check_for_birthday_collision(birthdays):
    """Checks for birthday collisions in the list."""
    seen = set()
    for birthday in birthdays:
        if birthday in seen:
            return True  # Collision found
        seen.add(birthday)
    return False  # No collision

def calculate_probability_of_collision(num_people):
    """Calculates the probability of a birthday collision logically."""
    if num_people > 365:
        return 1  # With more than 365 people, a collision is certain.
    
    probability_no_collision = 1
    for i in range(num_people):
        probability_no_collision *= (365 - i) / 365
    
    probability_collision = 1 - probability_no_collision
    return probability_collision

# Loop to find when probability exceeds 0.5
num_people = 1
while True:
    birthdays = generate_birthdays(num_people)
    
    # Check if there is a birthday collision
    if check_for_birthday_collision(birthdays):
        print(f"With {num_people} people, there is at least one pair sharing the same birthday.")
    
    # Calculate the theoretical probability of a birthday collision for num_people
    probability = calculate_probability_of_collision(num_people)
    
    # Print the probability for the current number of people
    print(f"Theoretical probability with {num_people} people: {probability:.4f}")
    
    if probability > 0.5:
        print(f"With {num_people} people, the probability exceeds 0.5.")
        break  # Stop when probability exceeds 0.5
    
    num_people += 1

Theoretical probability with 1 people: 0.0000
Theoretical probability with 2 people: 0.0027
Theoretical probability with 3 people: 0.0082
Theoretical probability with 4 people: 0.0164
Theoretical probability with 5 people: 0.0271
Theoretical probability with 6 people: 0.0405
Theoretical probability with 7 people: 0.0562
With 8 people, there is at least one pair sharing the same birthday.
Theoretical probability with 8 people: 0.0743
Theoretical probability with 9 people: 0.0946
Theoretical probability with 10 people: 0.1169
Theoretical probability with 11 people: 0.1411
Theoretical probability with 12 people: 0.1670
With 13 people, there is at least one pair sharing the same birthday.
Theoretical probability with 13 people: 0.1944
Theoretical probability with 14 people: 0.2231
Theoretical probability with 15 people: 0.2529
With 16 people, there is at least one pair sharing the same birthday.
Theoretical probability with 16 people: 0.2836
With 17 people, there is at least one pair shari